# 🚕 New York City Taxi Trip Duration 🚕

En esta práctica vamos a trabajar con el **dataset histórico de viajes en taxi amarillo de la ciudad de Nueva York**.

Este dataset es parte de una competencia original de Kaggle y está basado en los registros del **NYC Yellow Cab 2016**, publicados por la NYC Taxi and Limousine Commission (TLC).  
El objetivo es **predecir la duración de cada viaje** en segundos, usando variables relacionadas con la ubicación, la fecha y la cantidad de pasajeros.

---

## 📊 **Descripción de las columnas**

| Columna               | Descripción                                                                 |
|-----------------------|------------------------------------------------------------------------------|
| `id`                  | Identificador único de cada viaje.                                          |
| `vendor_id`           | Código del proveedor de taxi asociado al registro.                          |
| `pickup_datetime`     | Fecha y hora en la que se inició el viaje (cuando se activó el taxímetro).  |
| `dropoff_datetime`    | Fecha y hora en la que terminó el viaje (cuando se apagó el taxímetro).     |
| `passenger_count`     | Número de pasajeros (dato ingresado por el conductor).                      |
| `pickup_longitude`    | Longitud del punto de recogida.                                             |
| `pickup_latitude`     | Latitud del punto de recogida.                                              |
| `dropoff_longitude`   | Longitud del punto de destino.                                              |
| `dropoff_latitude`    | Latitud del punto de destino.                                               |
| `store_and_fwd_flag`  | Indica si el viaje se almacenó en memoria antes de enviarse al servidor.    |
| `trip_duration`       | Duración del viaje en segundos (variable objetivo para la predicción).      |

---

## 🎯 **Objetivo**

- Construir un **modelo de regresión supervisado** para predecir `trip_duration`.
- Registrar y comparar diferentes experimentos usando **MLflow** para aplicar prácticas básicas de **MLOps**.
- Demostrar cómo versionar modelos y parámetros de forma reproducible para un flujo de trabajo escalable.

---

## 🔗 **Fuente**

Este dataset se encuentra disponible públicamente en Kaggle: [NYC Taxi Trip Duration](https://www.kaggle.com/competitions/nyc-taxi-trip-duration)

---


#1) Setup e Imports

In [ ]:
# Instala MLflow
!pip install -q mlflow
!pip install -q pyngrok

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import mlflow
import mlflow.sklearn

#2) Cargar el Dataset

In [ ]:
df = pd.read_csv('train.csv')  # Asegúrate del nombre exacto

# Muestra un vistazo
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455.0
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663.0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124.0
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429.0
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556611 entries, 0 to 556610
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  556611 non-null  object 
 1   vendor_id           556611 non-null  int64  
 2   pickup_datetime     556611 non-null  object 
 3   dropoff_datetime    556611 non-null  object 
 4   passenger_count     556611 non-null  int64  
 5   pickup_longitude    556611 non-null  float64
 6   pickup_latitude     556611 non-null  float64
 7   dropoff_longitude   556611 non-null  float64
 8   dropoff_latitude    556610 non-null  float64
 9   store_and_fwd_flag  556610 non-null  object 
 10  trip_duration       556610 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 46.7+ MB


#3) Preprocesamiento Simple

In [ ]:
# Elimina filas con NaNs
df = df.dropna()

In [ ]:

cols = [
    'pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude',
    'passenger_count'
]

X = df[cols]
y = df['trip_duration']

# Siempre revisa si hay NaNs
print(X.isnull().sum())
print(y.isnull().sum())


pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64
0


In [ ]:

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Asegura float64 para evitar schema issues con MLflow
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

#4) Configura MLflow

In [ ]:
# Usaremos local tracking URI por defecto
mlflow.set_experiment("NYC_Taxi_Trip_Duration")

<Experiment: artifact_location='file:///content/mlruns/260240400766724174', creation_time=1752208284713, experiment_id='260240400766724174', last_update_time=1752208284713, lifecycle_stage='active', name='NYC_Taxi_Trip_Duration', tags={}>

# 5) Entrenamiento + Tracking

In [ ]:
def train_and_log_model(n_estimators, max_depth, X_train, X_test, y_train, y_test):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import mlflow
    import mlflow.sklearn

    with mlflow.start_run():
        # Modelo
        rf = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
        rf.fit(X_train, y_train)

        # Predicción y métrica
        y_pred = rf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        # Log params y métricas
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_metric("rmse", rmse)

        # input_example y signature
        input_example = X_train.iloc[:2]
        signature = mlflow.models.infer_signature(X_train, rf.predict(X_train))

        # Log model con Registry
        mlflow.sklearn.log_model(
            sk_model=rf,
            name="random_forest_model",
            input_example=input_example,
            signature=signature,
            registered_model_name="NYC_Taxi_Trip_Model"
        )

        print(f"Run finalizado con RMSE: {rmse:.2f}")

In [ ]:
# Primera corrida
train_and_log_model(100, 5, X_train, X_test, y_train, y_test)

Run finalizado con RMSE: 3295.59


Registered model 'NYC_Taxi_Trip_Model' already exists. Creating a new version of this model...
Created version '2' of model 'NYC_Taxi_Trip_Model'.


#6) Visualiza Runs

In [ ]:
# Para abrir la UI local en Colab, usa un túnel con ngrok
# 1. Inicia MLflow UI en segundo plano
get_ipython().system_raw("mlflow ui --port 5000 &")

# 2. Crea una cuenta y conecta ngrok (obten tu token unico)
# Ve a https://dashboard.ngrok.com/get-started/your-authtoken
# Copia tu token único (es una cadena larga, algo como 2NEb9XXX...)

from pyngrok import ngrok

# Abre el túnel
ngrok.set_auth_token("2gfrSPOOMcEFhJziAOkQ6SGFz3K_2mTzkKgbr8L6hRoTdowv1")  # reemplaza si lo tienes
public_url = ngrok.connect(5000)
print("MLflow UI:", public_url)


MLflow UI: NgrokTunnel: "https://71be63c4f70f.ngrok-free.app" -> "http://localhost:5000"


#7) Haz Varias Corridas

In [ ]:
# Otra corrida con otros hyperparams
n_estimators = 200
max_depth = 8

# Otra corrida con otros hyperparams
train_and_log_model(n_estimators, max_depth, X_train, X_test, y_train, y_test)

# cada run se guarda perfectamente versionado en tu Experimento y tu Model Registry

Run finalizado con RMSE: 3271.30


Registered model 'NYC_Taxi_Trip_Model' already exists. Creating a new version of this model...
Created version '3' of model 'NYC_Taxi_Trip_Model'.


In [ ]:
# Otra más
train_and_log_model(300, 10, X_train, X_test, y_train, y_test)

Run finalizado con RMSE: 3299.41


Registered model 'NYC_Taxi_Trip_Model' already exists. Creating a new version of this model...
Created version '4' of model 'NYC_Taxi_Trip_Model'.
